In [1]:
import os
import glob
import math
import pandas as pd
import ee
from tqdm import tqdm

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:

class SatelliteDataExtractor:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date

    def mask_s2_clouds(self, image):
        qa = image.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        return image.updateMask(mask).copyProperties(image, ['system:time_start'])

    def compute_s2_indices(self, image, variable):
        indices = {
            'NDVI': image.normalizedDifference(['B8', 'B4']).rename('NDVI'),
            'NDWI': image.normalizedDifference(['B3', 'B8']).rename('NDWI'),
            'NBR':  image.normalizedDifference(['B8', 'B12']).rename('NBR'),
            'MSI':  image.select('B11').divide(image.select('B8')).rename('MSI')
        }
        if variable in indices:
            return image.addBands(indices[variable])
        return image  # For direct bands

    def extract_sentinel2(self, lat, lon, variable, buffer=250):
        point = ee.Geometry.Point([lon, lat])
        region = point.buffer(buffer)

        image_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(self.start_date, self.end_date) \
            .filterBounds(region) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .map(self.mask_s2_clouds)

        if variable in ['NDVI', 'NDWI', 'NBR', 'MSI']:
            image_col = image_col.map(lambda img: self.compute_s2_indices(img, variable)).select(variable)
        else:
            image_col = image_col.select(variable)

        def extract_value(image):
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
            value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get(variable)
            return ee.Feature(None, {'date': date, variable: value})

        features = image_col.map(extract_value).filter(ee.Filter.notNull([variable]))
        return features.aggregate_array('date').getInfo(), features.aggregate_array(variable).getInfo()

    def add_layover_shadow_mask(self, image):
        dem = ee.Image('USGS/SRTMGL1_003')
        theta_i = image.select('angle')
        phi_i = ee.Terrain.aspect(dem).select('aspect')
        alpha_s = ee.Terrain.slope(dem).select('slope').multiply(math.pi / 180)
        phi_r = phi_i.multiply(math.pi / 180)
        theta_rad = theta_i.multiply(math.pi / 180)

        layover = phi_r.subtract(theta_rad).abs().lt(math.pi / 2).And(alpha_s.gte(theta_rad.tan().atan()))
        shadow = alpha_s.lte(theta_rad.tan().atan())
        mask = layover.Not().And(shadow.Not())
        return image.updateMask(mask)

    def extract_sentinel1(self, lat, lon, band, buffer=250):
        point = ee.Geometry.Point([lon, lat])
        region = point.buffer(buffer)

        def to_db(image):
            return ee.Image(10).multiply(image.log10()).copyProperties(image, ['system:time_start'])

        def preprocess(image):
            masked = self.add_layover_shadow_mask(image)
            return to_db(masked)

        s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filterDate(self.start_date, self.end_date) \
            .filterBounds(region) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
            .filter(ee.Filter.eq('resolution_meters', 10)) \
            .map(preprocess)

        s1_band = s1.select(band)

        def extract_value(image):
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
            value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get(band)
            return ee.Feature(None, {'date': date, band: value})

        features = s1_band.map(extract_value).filter(ee.Filter.notNull([band]))
        return features.aggregate_array('date').getInfo(), features.aggregate_array(band).getInfo()

    def extract_dem(self, lat, lon):
        point = ee.Geometry.Point([lon, lat])
        srtm = ee.Image('USGS/SRTMGL1_003')
        terrain = ee.Terrain.products(srtm)
        slope = terrain.select('slope').reduceRegion(ee.Reducer.mean(), point, scale=30).get('slope').getInfo()
        aspect = terrain.select('aspect').reduceRegion(ee.Reducer.mean(), point, scale=30).get('aspect').getInfo()
        return slope, aspect

def batch_extract(df, extractor, variable):
    results = []
    date_set = set()

    for _, row in tqdm(df.iterrows(), total=len(df)):
        lat, lon, network, station = row['latitude'], row['longitude'], row['network'], row['station']
        try:
            if variable in ['NDVI', 'NDWI', 'NBR', 'MSI', 'B5', 'B6', 'B7', 'B8', 'B2', 'B3', 'B4', 'B11', 'B12']:
                dates, values = extractor.extract_sentinel2(lat, lon, variable)
            elif variable in ['VV', 'VH']:
                dates, values = extractor.extract_sentinel1(lat, lon, band=variable)
            elif variable in ['slope', 'aspect']:
                slope, aspect = extractor.extract_dem(lat, lon)
                results.append({
                    'network': network,
                    'station': station,
                    'Latitude': lat,
                    'Longitude': lon,
                    'slope': slope,
                    'aspect': aspect
                })
                continue

            date_set.update(dates)
            results.append({
                'network': network,
                'station': station,
                'Latitude': lat,
                'Longitude': lon,
                'series': dict(zip(dates, values))
            })
        except Exception as e:
            print(f"Error: {station} → {e}")

    date_list = sorted(date_set)
    normalized = []
    for row in results:
        if 'series' not in row:
            normalized.append(row)
            continue
        record = {
            'network': row['network'],
            'station': row['station'],
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        }
        record.update({d: row['series'].get(d, float('nan')) for d in date_list})
        normalized.append(record)

    return pd.DataFrame(normalized)


In [ ]:

# Main execution loop
root_dir = 'data'
variables = ['NDVI', 'NDWI', 'NBR', 'MSI', 'B5', 'B6', 'B7', 'B8', 'B2', 'B3', 'B4', 'B11', 'B12', 'VV', 'VH', 'slope', 'aspect']
extractor = SatelliteDataExtractor(start_date='2016-06-01', end_date='2025-06-14')

for continent_folder in os.listdir(root_dir):
    continent_path = os.path.join(root_dir, continent_folder)
    if not os.path.isdir(continent_path) or continent_folder == 'satellite_data':
        continue

    input_csv_dir = os.path.join(continent_path, 'extracted_data', 'mean', 'agg_mean')
    if not os.path.exists(input_csv_dir):
        continue

    csv_files = glob.glob(os.path.join(input_csv_dir, '*.csv'))
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)

        for variable in variables:
            output_dir = os.path.join('data', 'satellite_data', continent_folder, variable)
            os.makedirs(output_dir, exist_ok=True)
            output_path = os.path.join(output_dir, os.path.basename(csv_file))

            if os.path.exists(output_path):
                print(f"✅ Skipped (already exists): {output_path}")
                continue

            print(f"🚀 Extracting {variable} for {csv_file}")
            df_result = batch_extract(df, extractor, variable)
            df_result.to_csv(output_path, index=False)
            print(f"✅ Saved: {output_path}")


### Modified code to correct the terrainc correction function- no values of VH and VV

In [2]:
import ee
import math
import pandas as pd
from tqdm import tqdm

class SatelliteDataExtractor:
    def __init__(self, start_date, end_date):
        self.start_date = start_date
        self.end_date = end_date

    def mask_s2_clouds(self, image):
        qa = image.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        return image.updateMask(mask).copyProperties(image, ['system:time_start'])

    def compute_s2_indices(self, image, variable):
        indices = {
            'NDVI': image.normalizedDifference(['B8', 'B4']).rename('NDVI'),
            'NDWI': image.normalizedDifference(['B3', 'B8']).rename('NDWI'),
            'NBR':  image.normalizedDifference(['B8', 'B12']).rename('NBR'),
            'MSI':  image.select('B11').divide(image.select('B8')).rename('MSI')
        }
        if variable in indices:
            return image.addBands(indices[variable])
        return image  # For direct bands

    def extract_sentinel2(self, lat, lon, variable, buffer=250):
        point = ee.Geometry.Point([lon, lat])
        region = point.buffer(buffer)

        image_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(self.start_date, self.end_date) \
            .filterBounds(region) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .map(self.mask_s2_clouds)

        if variable in ['NDVI', 'NDWI', 'NBR', 'MSI']:
            image_col = image_col.map(lambda img: self.compute_s2_indices(img, variable)).select(variable)
        else:
            image_col = image_col.select(variable)

        def extract_value(image):
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
            value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get(variable)
            return ee.Feature(None, {'date': date, variable: value})

        features = image_col.map(extract_value).filter(ee.Filter.notNull([variable]))
        return features.aggregate_array('date').getInfo(), features.aggregate_array(variable).getInfo()

    def compute_sar_variables(self, image, ref_angle_deg=35):
        dem = ee.Image("USGS/SRTMGL1_003")
        pi = ee.Number(math.pi)

        sigma0 = ee.Image.constant(10).pow(image.divide(10))
        theta_i = image.select('angle')
        alpha_s = ee.Terrain.slope(dem).select('slope')
        phi_s = ee.Terrain.aspect(dem).select('aspect')
        phi_i = ee.Terrain.aspect(theta_i).reduceRegion(
            reducer=ee.Reducer.mean(), geometry=image.geometry(), scale=1000, maxPixels=1e8
        ).get('aspect')
        phi_i = ee.Image.constant(phi_i)

        phi_r = phi_i.subtract(phi_s)
        theta_i_rad = theta_i.multiply(pi).divide(180)
        alpha_s_rad = alpha_s.multiply(pi).divide(180)
        phi_r_rad = phi_r.multiply(pi).divide(180)
        ref_rad = ee.Number(ref_angle_deg).multiply(pi).divide(180)

        alpha_r = alpha_s_rad.tan().multiply(phi_r_rad.cos()).atan()
        alpha_az = alpha_s_rad.tan().multiply(phi_r_rad.sin()).atan()
        theta_lia_rad = alpha_az.cos().multiply((theta_i_rad.subtract(alpha_r)).cos()).acos()
        theta_lia_deg = theta_lia_rad.multiply(180).divide(pi).rename("local_incidence_angle")

        vv_lin = sigma0.select('VV').rename('VV_sigma0')
        vh_lin = sigma0.select('VH').rename('VH_sigma0')
        vv_dB = image.select('VV').rename('VV_sigma0_dB')
        vh_dB = image.select('VH').rename('VH_sigma0_dB')

        vv_gamma = vv_lin.divide(theta_lia_rad.cos()).rename('VV_gamma0')
        vh_gamma = vh_lin.divide(theta_lia_rad.cos()).rename('VH_gamma0')

        c2_i = theta_lia_rad.cos().pow(2)
        c2_r = ref_rad.cos().pow(2)
        vv_gamma_norm = vv_gamma.multiply(c2_r).divide(c2_i).rename('VV_gamma0_norm')
        vh_gamma_norm = vh_gamma.multiply(c2_r).divide(c2_i).rename('VH_gamma0_norm')

        vh_vv_ratio = vh_lin.divide(vv_lin).multiply(c2_r).divide(c2_i)
        vh_vv_ratio_dB = ee.Image.constant(10).multiply(vh_vv_ratio.log10()).rename('VH_VV_ratio_norm_dB')

        return image.addBands([
            vv_lin, vh_lin, vv_dB, vh_dB,
            vv_gamma, vh_gamma, vv_gamma_norm, vh_gamma_norm,
            theta_lia_deg, vh_vv_ratio_dB
        ])

    def extract_sentinel1(self, lat, lon, band, buffer=250):
        point = ee.Geometry.Point([lon, lat])
        region = point.buffer(buffer)

        def preprocess(image):
            image = self.compute_sar_variables(image)
            return image.copyProperties(image, ['system:time_start'])

        s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
            .filterDate(self.start_date, self.end_date) \
            .filterBounds(region) \
            .filter(ee.Filter.eq('instrumentMode', 'IW')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
            .map(preprocess)

        s1_band = s1.select(band)

        def extract_value(image):
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
            value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get(band)
            return ee.Feature(None, {'date': date, band: value})

        features = s1_band.map(extract_value).filter(ee.Filter.notNull([band]))
        return features.aggregate_array('date').getInfo(), features.aggregate_array(band).getInfo()

    def extract_dem(self, lat, lon):
        point = ee.Geometry.Point([lon, lat])
        srtm = ee.Image('USGS/SRTMGL1_003')
        terrain = ee.Terrain.products(srtm)
        slope = terrain.select('slope').reduceRegion(ee.Reducer.mean(), point, scale=30).get('slope').getInfo()
        aspect = terrain.select('aspect').reduceRegion(ee.Reducer.mean(), point, scale=30).get('aspect').getInfo()
        return slope, aspect

def batch_extract(df, extractor, variable):
    results = []
    date_set = set()

    for _, row in tqdm(df.iterrows(), total=len(df)):
        lat, lon, network, station = row['latitude'], row['longitude'], row['network'], row['station']
        try:
            if variable in ['NDVI', 'NDWI', 'NBR', 'MSI', 'B5', 'B6', 'B7', 'B8', 'B2', 'B3', 'B4', 'B11', 'B12']:
                dates, values = extractor.extract_sentinel2(lat, lon, variable)
            elif variable in ['VV', 'VH', 'VV_sigma0', 'VH_sigma0', 'VV_sigma0_dB', 'VH_sigma0_dB', 'VV_gamma0', 'VH_gamma0', 'VV_gamma0_norm', 'VH_gamma0_norm', 'VH_VV_ratio_norm_dB', 'local_incidence_angle']:
                dates, values = extractor.extract_sentinel1(lat, lon, band=variable)
            elif variable in ['slope', 'aspect']:
                slope, aspect = extractor.extract_dem(lat, lon)
                results.append({
                    'network': network,
                    'station': station,
                    'Latitude': lat,
                    'Longitude': lon,
                    'slope': slope,
                    'aspect': aspect
                })
                continue

            date_set.update(dates)
            results.append({
                'network': network,
                'station': station,
                'Latitude': lat,
                'Longitude': lon,
                'series': dict(zip(dates, values))
            })
        except Exception as e:
            print(f"Error: {station} → {e}")

    date_list = sorted(date_set)
    normalized = []
    for row in results:
        if 'series' not in row:
            normalized.append(row)
            continue
        record = {
            'network': row['network'],
            'station': row['station'],
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        }
        record.update({d: row['series'].get(d, float('nan')) for d in date_list})
        normalized.append(record)

    return pd.DataFrame(normalized)


In [3]:
import os
import glob
import pandas as pd

# Initialization
root_dir = 'data'
output_root = os.path.join(root_dir, 'satellite_data')
start_date = '2016-06-01'
end_date = '2025-06-14'

variables = [
    'NDVI', 'NDWI', 'NBR', 'MSI', 'B5', 'B6', 'B7', 'B8', 'B2', 'B3', 'B4', 'B11', 'B12',  # Sentinel-2
    'VV_sigma0', 'VH_sigma0', 'VV_sigma0_dB', 'VH_sigma0_dB',                            # S1 Sigma
    'VV_gamma0', 'VH_gamma0', 'VV_gamma0_norm', 'VH_gamma0_norm',                       # Gamma normalized
    'VH_VV_ratio_norm_dB', 'local_incidence_angle',                                      # Additional S1 metrics
    'slope', 'aspect'                                                                    # DEM
]

extractor = SatelliteDataExtractor(start_date=start_date, end_date=end_date)

# Loop over continents and process each station file
for continent_folder in os.listdir(root_dir):
    continent_path = os.path.join(root_dir, continent_folder)
    if not os.path.isdir(continent_path) or continent_folder == 'satellite_data':
        continue

    input_csv_dir = os.path.join(continent_path, 'extracted_data', 'mean', 'agg_mean')
    if not os.path.exists(input_csv_dir):
        continue

    csv_files = glob.glob(os.path.join(input_csv_dir, '*.csv'))
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        for variable in variables:
            output_dir = os.path.join(output_root, continent_folder, variable)
            os.makedirs(output_dir, exist_ok=True)

            output_path = os.path.join(output_dir, os.path.basename(csv_file))
            if os.path.exists(output_path):
                print(f"✅ Skipped (already exists): {output_path}")
                continue

            print(f"🚀 Extracting {variable} for {csv_file}")
            df_result = batch_extract(df, extractor, variable)
            df_result.to_csv(output_path, index=False)
            print(f"✅ Saved: {output_path}")


✅ Skipped (already exists): data\satellite_data\africa\NDVI\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\NDWI\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\NBR\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\MSI\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B5\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B6\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B7\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B8\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B2\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B3\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B4\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B11\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_data\africa\B12\AMMA-CATCH.csv
✅ Skipped (already exists): data\satellite_